# Prueba de algoritmo
 Programa estructurado para el trading y procesos de analisis de Mercado
 
##Equal-Weight S&P 500 Index Fund 

In [ ]:
#Importar Librerias 

import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importar nuestra lista de Stock
    #La proxima cosa que nesecitamos hacer is importar la constituente de la S&P 500.
    #Esas  contituyentes cambian sobre tiempo, as en un ideal , pudiera directamente conectar aa indice provisto  jalar esas constituyente en tiempo real y regular las bases.
    #Jugando por el acceso al indice provisto API ista fuera de lado de la scopia del tema.
    # Guadar el CSV en la Carpeta donde esta el folio del software. y luego importarlo en Notebook

In [ ]:
#Import File
stocks = pd.read_csv('SP500.csv')
stocks

# Acquiring an API Token

    #Ahora es tiempo para importar nuestra IEX Cloud API Token. Esta es la data que proveeremos al momento de ser usada pensando en el tema.
    #Los API Tokens(  otras informaciones sensitivas) pudiera estar "Stored" en una (Secrect.py) ese archivo no esta obtenido en su repositoio local del git, estaremos usando un sandbox API token en este tema. Caules poromedios de la data que usamosremos es aleatorio-generado (mas Importanciones) no tieendra costo asociado con el 
    #Guadar en el mismo folio que se encuentra el software.
    

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

# Haciendo nuestra primera API Call

    #Ahora es tiempo de estructurar nuestra API Calls para IEX CLOUD
    
    #Nesecitamos la siguiente informacion acerca de la API:
    # Mercado de Capitalizacion for each Stock
    # Price of each Stock.

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)


# Parsing nuestra API 
    #La API Call eso lo ejecutamos in el pasado bloqueconteniendo toda la informacion requerida para cconstruir nuestra estrategia equal-weighr S&P 500. con esto dicho, no estara la data en el proyecto en un prosper formato aun. nesecitamos primero parse (ARLO)

In [ ]:
price = data['lastPrice']    #dependiendo a los caracteres quese quiere analizar
market_cap = data['marketCap']
#print(market_cap/10000000000)

# Agregando Nuestra data de stocks para un DataFrame de Pandas 

    # Luego nesecitaremos hacer es agregar nuestro precio del stock  y capitalizacion del mercado para un DataFrame de Pandas. Pensar una DataFrame que equivalga a la version de python de un spreedsheet, esta tiendas.
    #
    

In [ ]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'NUmber of shares to buy']
final_dataframe = pd.DataFrame([[0,0,0,0]], columns = my_columns)
final_dataframe

In [ ]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index = my_columns
    ),
    ignore_index=True
)

# Looping Through the Ticker in Our List of stocks

    #Using the same logic that outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a for loop

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox_iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns),
    ignore_index = True                          
    )

In [ ]:
final_dataframe

# Using Bach  API Calls to improveperformance
    
    #Bach API Calls son una de las maneras faciles para probar la performace del codigo 
    #Esta es porque el HTTP requests son tipicalmente unos de los componentes mas lentos de un script
    # Tambiem, API proveera algunas obtenciones la tasa de descuento por usar el Bach API call desde ellos son mas facil pot la API providiendo responder para IEX Cloud sus limitantes del Bach API call para 100 tickers por requests. 

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups(i)))
 #  print(symbol_strings(i))
final_dataframe = pd.DataFrame(columns = my_colums)

for symbol_strings in symbol_strings:
    batch_api_call_url = f'https://sandbox_iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json
    for symbol_strings in symbol_string.split(','):
          final_dataframe = final_dataframe.append(
                pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True                          
        )

final_dataframe


# Calculating the number of shares to Buy

In [ ]:
portfolio_size = input('Enter the value of your portfolio')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \nPlease try again: ")
    portfolio_size = input ('Enter the value of your portfolio: ')
    val = float(portfolio_size)

In [ ]:
#portafolio_size
position_size = val/len(final_dataframe.index)
#print(position_size)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor[position_size/final_dataframe.loc[i, 'Stock Price']]
number_of_apple_shares = position_size / 500
print(number_of_apple_shares)


# Formating our Excel Output

    aqui realizaremos la la libreria de xlsxwriter para creararchivos de formato xlsx

In [ ]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

# Crear los formatos que nesecitaremos para nuestra xlsxwriter

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        'bg_color' : background_color,
        'border':
    }

)

dollar_format = writer.book.add_format(
    {
        'num_format' : '$0,00',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border':
    }

)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_color,
        'bg_color' : background_color,
        'border':
    }

)

# Ampliar los formatos para las columnasde nuestro .xlsxwFile

In [ ]:
#writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
#writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
#writer.save

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'NUmber of Shares to Buy', integer_format)


In [ ]:
column_format = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['NUmber of Shares to Buy', integer_format]
}

for column in column_format.keys():
    writer.sheet['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheet['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1]


In [ ]:
 writer.save()